In [ ]:
from classes.Dataset_Basic import Dataset_Basic
from classes.Process_Stages import Process_Stages

from classes.Dataset_Settings import Dataset_Settings

from run_models.cosine_sililarity.classes.Dataset_Cosine import Dataset_Cosine
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

from run_models.embed_words.classes.Embed_Words import Embed_Words

from run_models.gensim.services.gensim_services import gensim_download, gensim_save, gensim_load
from word_embedding.models.classes.EmbeddingModel import EmbeddingModel

name_dataset = "conceptnet_cosine_similarity"

datasets = {}
datasets["neural_course"] = Dataset_Cosine(
    df_name="neural_course",
    model_name=name_dataset, # used for dir name inside data_saved

    language="english",

    datasets = {
        "emebed_words_conceptnet": Dataset_Settings(
            df=None,
            may_run_now=False,
            required=True,
            parquet=True
        ),
        name_dataset: Dataset_Settings(
            df=None,
            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="gensim"
        ),
    },

    columns_to_add = {name_dataset: {"cosine_similarity": []}},

)

# get dataset, process dataset, save dataset
datasets["neural_course"].get_dataset()
datasets["neural_course"].process_dataset()
datasets["neural_course"].add_columns()
datasets["neural_course"].save()


# BERT embedding model

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Initialize the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Your sentence
sentence = "This is a simple sentence."

# Tokenize the sentence and encode it to IDs
input_ids = tokenizer.encode(sentence, add_special_tokens=True)

# Create tensors
input_ids = torch.tensor([input_ids])

# Forward pass: compute the BERT embeddings
with torch.no_grad():
    outputs = model(input_ids)

# Extract the last hidden states (the embeddings)
last_hidden_states = outputs[0]

# # Print the embeddings
# for i, token_str in enumerate(tokenizer.convert_ids_to_tokens(input_ids[0])):
#     print(token_str, last_hidden_states[0, i])



In [ ]:
def tokenize_sentence(sentence):
    # Tokenize the sentence and encode it to IDs
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)

    # Create tensors
    input_ids = torch.tensor([input_ids])

    # Forward pass: compute the BERT embeddings
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract the last hidden states (the embeddings)
    last_hidden_states = outputs[0]

    return last_hidden_states

In [ ]:
# classes
from data.embed_words.BERT_embedding.classes.BERT_Embedding import BERT_Embedding
from classes.Dataset_Settings import Dataset_Settings

BERT_embed = BERT_Embedding(
    df_name="neural_course",
    model_name="BERT", # used for dir name inside data_saved

    language="english",

    datasets = {
        "standardized_splits": Dataset_Settings(
            df=None,
            df_name="splits",
            base_dir="data",

            may_run_now=False,
            required=True,
        ),
        "BERT": Dataset_Settings(
            df=None,
            df_name="BERT",
            base_dir="data/embed_words/data",

            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="standardized_splits",
            force_run=False
        ),
    },

    pre_trained_BERT_model_name="bert-base-uncased"

)

BERT_embed.run_all()


In [ ]:
"Marly is lief"

In [ ]:
# embed a sentence in BERT_tokenized
import pandas as pd

df_BERT = pd.read_parquet("data/embed_words/data/BERT/data/neural_course.parquet")
df_2 = pd.read_csv("data/splits/data/neural_course.csv")
fast_text = pd.read_parquet("data/embed_words/data/fasttext/data/neural_course.parquet")

basic = pd.read_csv("data/basic_processed/data/neural_course.csv")


In [ ]:
from services.string_array import str_to_array
from scipy.spatial.distance import cosine

1 - cosine(str_to_array(df_BERT.iloc[0]["student_answer"])[0], str_to_array(df_BERT.iloc[0]["student_answer"])[0])


In [16]:
from performance_tracking.classes.Dataset_api import Dataset_api

dataset = Dataset_api(
    dir="data/splits/data",
    file_name="ASAP_sas",
    seed=42,
    shots=3 # DO NOT CHANGE UNTILL FINISHED WITH RUN THROUGH ALL DATASETS!
)

dataset.split_datasets()


Found ASAP_sas with type csv in (data/splits/data) and converted it to df
fill_nan_values


In [21]:
dataset["validation"].head(20)



,row_id,question_id,student_answer,assigned_points,question,reference_answer,domain,max_points,dataset_name,normalized_points,level_1,student_answer_1,reference_answer_1,assigned_points_1,student_answer_2,reference_answer_2,assigned_points_2,student_answer_3,reference_answer_3,assigned_points_3
0,6248,214,The author of the article compares China's pan...,2,NaN,Pandas in China and koalas in Australia are si...,english_language_arts,2.0,ASAP_sas,1.000000,0,Koalas eat nothing but leaves and pandas eat n...,Pandas in China and koalas in Australia are si...,1,The pandas and koala bears there native to the...,Pandas in China and koalas in Australia are si...,1,The pandas in China are equivalent to the koal...,Pandas in China and koalas in Australia are si...,1
1,15515,217,Difusion is where gases move across the cell f...,2,NaN,Cells use many types of transport to move subs...,biology,3.0,ASAP_sas,0.666667,0,"Osmosis, Diffusion and Active Transport.",Cells use many types of transport to move subs...,0,The cell wall processes the movement in and ou...,Cells use many types of transport to move subs...,0,1.anaphase so anaphase means the chromosomes l...,Cells use many types of transport to move subs...,0
2,23876,220,The author uses heading in alazised bold font ...,1,NaN,The author organizes the article by putting th...,english,2.0,ASAP_sas,0.500000,0,The author categorizes this article. For examp...,The author organizes the article by putting th...,2,He divides the article into smaller subarticle...,The author organizes the article by putting th...,1,The author organizes this article in a way tha...,The author organizes the article by putting th...,2
3,4034,213,a) Based on the students data I've concluded t...,2,NaN,"Based on the student's data, plastic A stretch...",science,3.0,ASAP_sas,0.666667,0,(A) A conclusion based on the student's data i...,"Based on the student's data, plastic A stretch...",1,Based on the data I can include that plastic B...,"Based on the student's data, plastic A stretch...",3,Based on the student's data plastic type B str...,"Based on the student's data, plastic A stretch...",2
4,411,212,The additional info I would need to replicate ...,0,NaN,"In order to replicate this experiment, I would...",science,3.0,ASAP_sas,0.000000,0,What temp is everything? they should include t...,"In order to replicate this experiment, I would...",2,1) I would need to know how much vinegar is be...,"In order to replicate this experiment, I would...",1,Additional information you would need would be...,"In order to replicate this experiment, I would...",0
5,3735,213,1)My conclusion would be that plastic type D s...,1,NaN,"Based on the student's data, plastic A stretch...",science,3.0,ASAP_sas,0.333333,0,(A) A conclusion based on the student's data i...,"Based on the student's data, plastic A stretch...",1,Based on the data I can include that plastic B...,"Based on the student's data, plastic A stretch...",3,Based on the student's data plastic type B str...,"Based on the student's data, plastic A stretch...",2
6,5932,214,Pandas in China and Koalas in Australia are bo...,2,NaN,Pandas in China and koalas in Australia are si...,english_language_arts,2.0,ASAP_sas,1.000000,0,Koalas eat nothing but leaves and pandas eat n...,Pandas in China and koalas in Australia are si...,1,The pandas and koala bears there native to the...,Pandas in China and koalas in Australia are si...,1,The pandas in China are equivalent to the koal...,Pandas in China and koalas in Australia are si...,1
7,9378,215,The significance of the word invasive is somet...,0,NaN,The significance of the word invasive is that ...,english_language_arts,2.0,ASAP_sas,0.000000,0,The article talks about certain species that a...,The significance of the word invasive is that ...,1,The signifance of the word invasive is it stat...,The significance of the word invasive is that ...,0,"Invasive invading researchers find pythons ""in...",The significance of the word invasive is that ...,1
8

In [22]:
nan_rows = dataset["validation"][dataset["validation"]['reference_answer_3'].isna()]
print(nan_rows)


Empty DataFrame
Columns: [row_id, question_id, student_answer, assigned_points, question, reference_answer, domain, max_points, dataset_name, normalized_points, level_1, student_answer_1, reference_answer_1, assigned_points_1, student_answer_2, reference_answer_2, assigned_points_2, student_answer_3, reference_answer_3, assigned_points_3]
Index: []


In [3]:
import pandas as pd

# create sample dataset
sample = pd.read_csv("data/splits/data/concatenated_domains.csv")
sample = sample.sample(100, random_state=42)


In [4]:
sample.head(10)

,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points
14266,19976,NaN,219,Mr. Leonard has poor reading skills and therfo...,The background information given allows Paul t...,0,2.0,english,ASAP_sas,0.0
17958,26067,NaN,221,black :: the color black heats the doghouse t...,white :: The best paint color for the doghous...,2,2.0,science,ASAP_sas,1.0
4197,3040,NaN,213,a) Based on the students data plastic type B s...,"Based on the student's data, plastic A stretch...",3,3.0,science,ASAP_sas,1.0
18428,26537,NaN,221,white :: the dog house should be white because...,white :: The best paint color for the doghous...,2,2.0,science,ASAP_sas,1.0
5572,5267,NaN,214,Pandas and koalas are similar because the both...,Pandas in China and koalas in Australia are si...,1,2.0,english_language_arts,ASAP_sas,0.5
10333,12448,NaN,216,Four major steps involved in protein synthesis is,"First, the mRNA attaches to a ribosome and the...",0,3.0,biology,ASAP_sas,0.0
1272,1272,What is a stack?,175,an object list that stores elements in a parti...,"A data structure that can store elements, whic...",5,5.0,science,texas,1.0
5426,5121,NaN,214,China's panda bears are similar to Australia's...,Pandas in China and koalas in Australia are si...,0,2.0,english_language_arts,ASAP_sas,0.0
7477,8433,NaN,215,"The word ""invasive"" has greater significance o...",The significance of the word invasive is that ...,2,2.0,english_language_arts,ASAP_sas,1.0
8958,11073,NaN,216,well the organism is going to get sick but the...,"First, the mRNA attaches to a ribosome and the...",0,3.0,biology,ASAP_sas,0.0


In [5]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train (70%) and the remaining data (30%)
train, temp_df = train_test_split(sample, test_size=0.3, random_state=42)

# Split the remaining data further into test (20% of the original dataset) and validation (10% of the original dataset) sets
test, validation = train_test_split(temp_df, test_size=1/3, random_state=42)


In [6]:
validation.iloc[0:].shape[0]


10

In [7]:
from services.save import save

save(
    dir="testing/data", 
    file_name="sample", 
    df=sample
)


In [8]:
sample.query("question_id == 8")


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points


In [12]:
from performance_tracking.classes.Dataset_api import Dataset_api
from performance_tracking.classes.Dataset import Dataset

dataset = Dataset_api(
    dir="data/splits/data",
    file_name="beetle",
    seed=42,
    shots=3, # DO NOT CHANGE UNTILL FINISHED WITH RUN THROUGH ALL DATASETS!
    left_out_dataset="biology"
)

dataset.split_datasets()


Found beetle with type csv in (data/splits/data) and converted it to df
fill_nan_values


In [14]:
dataset["validation"]


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points,level_1,student_answer_1,reference_answer_1,assigned_points_1,student_answer_2,reference_answer_2,assigned_points_2,student_answer_3,reference_answer_3,assigned_points_3
0,747,What is the relationship between the voltage r...,63,the difference between the voltage terminals.,Voltage is the difference in electrical states...,0,1,NaN,beetle,0.0,0.0,the voltage reading is the difference between ...,Voltage is the difference in electrical states...,0.0,if the voltage reading is 0 volts then the bul...,Voltage is the difference in electrical states...,0.0,The terminal.,Voltage is the difference in electrical states...,0.0
1,6388,Explain your reasoning.,128,b and c are parallel to a,bulbs B and C are still in closed paths with t...,1,1,NaN,beetle,1.0,0.0,bulb A is not contained in the same circuit as...,bulbs B and C are still in closed paths with t...,0.0,Bulb B and C are on different closed paths con...,bulbs B and C are still in closed paths with t...,1.0,there is still a closed path that contains the...,bulbs B and C are still in closed paths with t...,1.0
2,1310,Explain why you got a voltage reading of 1.5 f...,68,the terminal is connected to the battery.,Terminal 2 and the positive terminal are separ...,0,1,NaN,beetle,0.0,0.0,The positive battery terminal is seperated by ...,Terminal 2 and the positive terminal are separ...,1.0,termainal2,Terminal 2 and the positive terminal are separ...,0.0,the postive battery is seperated by a gap from...,Terminal 2 and the positive terminal are separ...,1.0
3,2053,Why was bulb C off when switch Z was open?,77,because bulb C was separated by a gap from the...,There was no longer a closed path containing B...,1,1,NaN,beetle,1.0,0.0,Bulb C is not contained in the in the circuit.,There was no longer a closed path containing B...,0.0,Because switch Y was open and bulb C is not in...,There was no longer a closed path containing B...,1.0,becuase bulb C will be taken off the closed pa...,There was no longer a closed path containing B...,1.0
4,4591,Explain why circuit 2 is not a short circuit.,106,The circuit is not closed around the battery.,The battery in 2 is not in a closed path,0,1,NaN,beetle,0.0,0.0,the battery is not on a closed path,The battery in 2 is not in a closed path,1.0,"the battery terminals don""t connect without an...",The battery in 2 is not in a closed path,0.0,the battery is not in a closed bath alone,The battery in 2 is not in a closed path,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,5501,Explain your reasoning.,118,different paths with the battery,B and C are in the same path with the battery,0,1,NaN,beetle,0.0,0.0,bulb b and bulb c are contained in the same pa...,B and C are in the same path with the battery,1.0,contained within the same path with the battery,B and C are in the same path with the battery,1.0,both bulbs are on the same path with the battery,B and C are in the same path with the battery,1.0
658,737,What is the relationship between the voltage r...,63,if there is a measurement reading then the ele...,Voltage is the difference in electrical states...,0,1,NaN,beetle,0.0,0.0,the voltage reading is the difference between ...,Voltage is the difference in electrical states...,0.0,if the voltage reading is 0 volts then the bul...,Voltage is the difference in electrical states...,0.0,The terminal.,Voltage is the difference in electrical states...,0.0
659,1374,Explain why you got a voltage reading of 1.5 f...,69,terminal 2 is not connected to the positive te...,Terminal 2 and the positive terminal are not c...,1,1,NaN,beetle,1.0,0.0,the positive battery is seperated by a gap fro...,Terminal 2 and the positive terminal are not c...,1.0,the positive terminal is separated by a gap fr...,Terminal 2 and the positive terminal are not c...,1.0,the terminal is connected to the battery.,Terminal 2 and the posi

In [2]:
len(dataset["validation"])


8

In [ ]:
dataset["validation"].head(2)


# Results

In [ ]:
performance = pd.read_csv("performance_tracking/tracking/performance_tracking.csv")
performance = performance.query("dataset_split == 'validation_df'")
performance = performance.query("dataset_name != 'concatenated_datasets'")
performance = performance.query("dataset_name != 'concatenated_domains'")


In [ ]:
performance.head(2)

In [ ]:

grouped_column = "embedding_model_name"

# embedding_model_name classification_model_name
performance_embed_avg = performance.groupby(grouped_column).mean().reset_index()
performance_embed_avg_classification = performance_embed_avg.loc[:, [grouped_column, "accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]]
performance_embed_avg_classification = performance_embed_avg_classification.round(2)

performance_embed_avg_cont = performance_embed_avg.loc[:, [grouped_column, "rmse", "pears_correlation", "p_value"]]
performance_embed_avg_cont = performance_embed_avg_cont.round(2)

performance_embed_std = performance.groupby(grouped_column).std().reset_index()
performance_embed_std_classification = performance_embed_std.loc[:, [grouped_column, "accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]]
performance_embed_std_classification = performance_embed_std_classification.round(2)

performance_embed_std_cont = performance_embed_std.loc[:, [grouped_column, "rmse", "pears_correlation", "p_value"]]
performance_embed_std_cont = performance_embed_std_cont.round(2)


In [ ]:
print(performance_embed_avg_classification.to_latex(index=False))





In [ ]:
print(performance_embed_avg_cont.to_latex(index=False))


In [ ]:
print(performance_embed_std_classification.to_latex(index=False))


In [ ]:
print(performance_embed_std_cont.to_latex(index=False))

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-large-uncased')

# Sentence for which we want to create embeddings
sentence = "This is a sample sentence."

# Tokenize the sentence and convert to input IDs.
inputs = tokenizer(sentence, return_tensors='pt')

# Run the sentence through the model to get the embeddings.
with torch.no_grad():
    outputs = model(**inputs)

# We get a tuple as output. The first element of the tuple is the embeddings.
embeddings = outputs[0]

print(embeddings)


In [ ]:
len(embeddings[0])

In [ ]:
inputs

In [ ]:
from transformers import BertModel, BertTokenizer

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

# A very long sentence
sentence = "This is a very long sentence..." # Assume this is a very long sentence

# Tokenize the sentence and get the tokens
tokens = tokenizer.tokenize(sentence)

# Define the maximum chunk size (max BERT length - 2 for [CLS] and [SEP])
max_chunk_size = 510

# Split the tokens into chunks
chunks = [tokens[i:i + max_chunk_size] for i in range(0, len(tokens), max_chunk_size)]

# For each chunk, add special tokens, convert to input IDs, feed to BERT, and get embeddings
for chunk in chunks:
    # Add special tokens
    chunk = ['[CLS]'] + chunk + ['[SEP]']
    # Convert to input IDs
    inputs = tokenizer.convert_tokens_to_ids(chunk)
    inputs = torch.tensor([inputs])  # Add batch dimension
    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs)
    # The first element of outputs is the embeddings
    embeddings = outputs[0]
    print(embeddings)


In [ ]:
from performance_tracking.classes.Performance_Row import Performance_Row


In [ ]:
row = Performance_Row(
    # id what is being tracked
    dataset_name="test",
    embedding_seperated=self.measurement_settings.embedding_seperated,
    embedding_model_name = self.measurement_settings.embedding_model_name,
    sentence_embedding_method = self.measurement_settings.sentence_embedding_method,
    feature_engenearing_method = self.measurement_settings.feature_engenearing_method,
    grading_model = self.measurement_settings.grading_model,
    seed_data_split = self.measurement_settings.seed_data_split,
    shots=self.shots,
    epochs=self.epochs,

    dataset_split=dataset_split,

    # duplicates handeling
    settings_performance_tracking=self.measurement_settings.settings_performance_tracking
)


